In [ ]:
import pandas as pd
from datetime import timedelta, datetime
import os

In [ ]:
ITERATIONS = 6

In [ ]:

def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

In [ ]:
base_path = '../results/ce/ce_6h_2301132259_1'
run_dir = 'ce_6h_2301132259_1'
run_name = 'ce_6h_2301132259'

In [ ]:
res = '6h'

In [ ]:
x = 'ce_12h_2301142318_1'


In [ ]:
winners = get_winner_list(base_path)

In [ ]:
dirs = get_immediate_subdirectories('../results/ce/')
for dir in dirs:
    print(dir)
    ds = dir.split('_')
    res = ds[1]
    run = dir[0:dir.rindex('_')]
    base_path = f'../results/ce/{dir}'
    winners = get_winner_list(base_path)
    c_state = concat_state_files(winners, base_path, run, res)
    c_actions = concat_actions_files(winners, base_path, run, res)

In [ ]:
c_state = concat_state_files(winners, base_path, run_name, res)
c_actions = concat_actions_files(winners, base_path, run_name, res)

In [ ]:
def get_delta_fix(negative=False):
    hours = 24
    if res == '6h':
        hours = 6
    if res == '12h':
        hours = 12
    if res == '1h':
        hours = 1
    if negative:
        hours *= -1
    return timedelta(hours=hours)

def get_winner_list(base_folder):
    return pd.read_csv(f"{base_folder}/all_stats.csv")['winner'].tolist()
    
def concat_state_files(winner_models, base_path, run, res, iterations=6):
    # concat separate state files from each iteration into one file for the whole run
    states = pd.DataFrame()
    for iteration in range(iterations):
        filename = f"{base_path}/{run}_iter{iteration}_ensemble_state.csv"
        df_state_iter = pd.read_csv(filename, index_col=0)

        df_state_iter['date'] = df_state_iter.index
        df_state_iter = df_state_iter.reset_index(drop=True)
        df_state_iter.loc[0, 'date'] = (pd.to_datetime(df_state_iter.loc[1]['date']) + get_delta_fix(negative=True)).strftime("%Y-%m-%d" if res == '1d' else "%Y-%m-%d %X")
        df_state_iter = df_state_iter.set_index('date', drop=True)
        df_state_iter['iteration'] = iteration
        df_state_iter['model'] = winner_models[iteration]

        if states.empty:
            states = df_state_iter
        else:
            states = pd.concat([states, df_state_iter])

    # return states
    states.to_csv(f"{base_path}/all_state.csv")
    
def concat_actions_files(winner_models, base_path, run, res, iterations=6, stock_dim=10):
    # concat separate actions files from each iteration into one file for the whole run
    actions = pd.DataFrame()
    for iteration in range(iterations):

        filename = f"{base_path}/{run}_iter{iteration}_ensemble_actions.csv"
        df_actions_iter = pd.read_csv(f"{filename}")

        # add last day for each iteration
        last_date = (pd.to_datetime(df_actions_iter.iloc[-1]['date']) + get_delta_fix()).strftime("%Y-%m-%d" if res == '1d' else "%Y-%m-%d %X")
        last_row = pd.DataFrame([[last_date] + [0] * stock_dim + [-1] * stock_dim])
        last_row.columns = df_actions_iter.columns
        df_actions_iter = pd.concat([df_actions_iter, last_row])
        df_actions_iter['iteration'] = iteration
        df_actions_iter['model'] = winner_models[iteration]
        if actions.empty:
            actions = df_actions_iter
        else:
            actions = pd.concat([actions, df_actions_iter])
        # print(results_base)

    actions = actions.reset_index(drop=True)
    target_filename = f"{base_path}/all_actions.csv"
    actions.to_csv(target_filename, index=False)